In [139]:
# Binance API information
API_KEY = ""
SECRET_KEY = ""


In [42]:
import requests
import json
import pandas as pd
import datetime as dt

In [190]:
import requests
import time

def fetch_binance_historical_data(symbol, interval, start_time, end_time):
    """
    Fetches historical k-line data from Binance API for a given symbol, interval, start time, and end time.
    
    :param symbol: str, trading pair symbol (e.g., 'BTCUSDT' for Bitcoin/USDT)
    :param interval: str, timeframe for the historical data (e.g., '1d' for daily)
    :param start_time: int, start timestamp in milliseconds
    :param end_time: int, end timestamp in milliseconds
    :return: list of dictionaries, historical k-line data
    """
    # Define API endpoint
    endpoint = 'https://api.binance.com/api/v3/klines'
    
    # Initialize empty list to store historical data
    historical_data = []
    
    # Loop until end time is reached
    while start_time < end_time:
        # Calculate remaining time until end time or maximum allowed timeframe (500 data points)

        if interval[-1] == 'h':
            interval_size = (int(interval[0]) * 60) - 1
        else:
            interval_size = int(interval[:-1])
        
        remaining_time = min(end_time - start_time, 500 * interval_size * 60 * 1000)
        
        # Define API parameters for current request
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'endTime': start_time + remaining_time,
            'limit': 500,
        }
        
        # Send API request and parse response
        response = requests.get(endpoint, params=params)
        data = response.json()

        # display(len(data)) 
        
        # Append fetched data to historical_data list
        historical_data.extend(data)
        
        # Update start time for next request
        start_time += remaining_time + (int(interval[:-1]) * 60 * 1000)
        
        # Add delay to avoid hitting API rate limits (optional)
        time.sleep(1)
    
    return historical_data


In [218]:
# hist_data = fetch_binance_historical_data(symbol = 'AXSUSDT',
#                                           interval = '1h',
#                                           start_time = int(dt.datetime(2023,3,1).timestamp()*1000),
#                                           end_time = int(dt.datetime(2023,4,13).timestamp()*1000))

hist_data = fetch_binance_historical_data(symbol = 'AXSUSDT',
                                          interval = '1h',
                                          start_time = int(dt.datetime(2023,3,1).timestamp()*1000),
                                          end_time = int(dt.datetime(2023,5,6).timestamp()*1000))

In [219]:
len(hist_data)

1584

In [220]:
data = pd.DataFrame(hist_data)
#format columns name
data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
data=data.astype(float)
data.tail()

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2023-05-05 20:00:00,1.683288e+12,7.62,7.64,7.47,7.55,44559.89,1.683292e+12,336678.8271,1578.0,14863.18,112301.7937,0.0
2023-05-05 21:00:00,1.683292e+12,7.55,7.62,7.53,7.61,16389.72,1.683295e+12,124179.4459,484.0,9290.71,70441.7859,0.0
2023-05-05 22:00:00,1.683295e+12,7.62,7.69,7.61,7.68,36715.20,1.683299e+12,280721.0257,975.0,20826.63,159304.9533,0.0
2023-05-05 23:00:00,1.683299e+12,7.68,7.73,7.67,7.71,30138.36,1.683302e+12,232078.7388,757.0,20587.24,158573.0048,0.0
2023-05-06 00:00:00,1.683302e+12,7.72,7.81,7.71,7.77,70995.83,1.683306e+12,551780.6399,1755.0,44215.81,343750.4254,0.0


In [129]:
import numpy as np
import pandas as pd


def get_events(close, h, l, max_holding_period=10):
    """
    Implements the Triple Barrier method for labelling financial data.

    :param close: A pandas series of closing prices.
    :param h: The upper barrier as a positive float or integer.
    :param l: The lower barrier as a negative float or integer.
    :param max_holding_period: The maximum number of days to hold a position.
    :return: A pandas dataframe of labelled events.
    """
    # Compute vertical barrier
    t_events = []
    for date in close.index:
        idx = close.index.get_loc(date)
        try:
            close_price = close.iloc[idx]
            ahead = close.index[idx + 1: idx + max_holding_period + 1]
            events = ahead[(close.loc[ahead] >= close_price * h) | (close.loc[ahead] <= close_price * l)]
            t_events.extend([(date, x) for x in events])
        except IndexError:
            print("Something just happened")
            pass
    t_events = pd.MultiIndex.from_tuples(t_events)
    # Remove duplicated index
    t_events = t_events.to_series().groupby(level=0).first()
    # Compute horizontal barrier
    upper = close * h
    lower = close * l
    # Set vertical barrier
    events = pd.DataFrame({'t1': t_events}, index=t_events)
    prices = pd.DataFrame({'price': close, 'upper': upper, 'lower': lower}, index=close.index)
    events['t1'] = events['t1'].apply(lambda x: prices.index.searchsorted(x))
    # Apply label

    # Calculate returns between t0 and t1
    if not prices.index.is_unique:
        raise ValueError("Index must be unique")
        
    t1 = events['t1'].values
    # t1 = t1[t1 < prices.index[-1]]
    t1 = pd.Series(t1, index=events.index[:len(t1)])

    print(len(t1))
    print(max(t1))
    print(min(t1))
    print(len(prices['price']))
        
    events['pt'] = (prices['price'].iloc[t1].values / prices['price'].values - 1) * 100
    # print(len(prices['price'].values))
    # events['pt'] = (prices['price'].iloc[events['t1'].values].values / prices['price'].values - 1) * 100
    print(events)
    events['target'] = np.where(events['pt'] > 0, 1, -1)
    return events

# Compute labels
events = get_events(data['close'], h=1.02, l=0.98, max_holding_period=10)

# Print events
print(events)

558
1032
1032
1032


IndexError: positional indexers are out-of-bounds

In [177]:
import pandas as pd
import numpy as np
import ta
import yfinance as yf

# Load data
symbol = "AAPL"
start_date = "2020-01-01"
end_date = "2022-05-03"
ohlcv = yf.download(symbol, start=start_date, end=end_date)

# Define parameters
t_final = 10  # final holding period
upper_lower_multipliers = [2, 2]  # multiplier for top/bottom barriers

# Compute daily volatility and moving averages
daily_volatility = ta.volatility.BollingerBands(
    ohlcv["Close"], window=20, window_dev=2
).bollinger_mavg() / np.sqrt(252)

sma = ta.trend.sma_indicator(ohlcv["Close"], window=20)

# Resample OHLCV data to daily frequency
data_daily = ohlcv.resample('D').agg({'Open': 'first',
                                     'High': 'max',
                                     'Low': 'min',
                                     'Close': 'last',
                                     'Volume': 'sum'})

prices = data_daily['Close']


def get_barriers(prices, daily_volatility, t_final, upper_lower_multipliers):
    # create a container
    barriers = pd.DataFrame(
        columns=[
            "days_passed",
            "price",
            "vert_barrier",
            "top_barrier",
            "bottom_barrier",
        ],
        index=daily_volatility.index,
    )
    # vert_barrier = []
    # top_barrier = []
    # bottom_barrier = []

    for day, vol in daily_volatility.iteritems():
        days_passed = len(
            daily_volatility.loc[daily_volatility.index[0] : day]
        )
        # print(days_passed)
        # print("days passed {}".format(days_passed))
        # print("t final {}".format(t_final))
        # set the vertical barrier

        try:
            if (days_passed + t_final < len(daily_volatility.index) and t_final != 0):
                vert_barrier = daily_volatility.index[days_passed + t_final]
                # vert_barrier = (daily_volatility).tolist()[days_passed + t_final]
            else:
                vert_barrier = np.nan

        except:
            print(days_passed)

        # set the top barrier
        if upper_lower_multipliers[0] > 0:
            top_barrier = prices.loc[day] + prices.loc[day] * upper_lower_multipliers[
                0
            ] * vol
        else:
            # set it to NaNs
            top_barrier = pd.Series(index=prices.index)
        # set the bottom barrier
        if upper_lower_multipliers[1] > 0:
            bottom_barrier = prices.loc[day] - prices.loc[day] * upper_lower_multipliers[
                1
            ] * vol
        else:
            # set it to NaNs
            bottom_barrier = pd.Series(index=prices.index)

        barriers.loc[day, ["days_passed", "price", "vert_barrier", "top_barrier", "bottom_barrier"]] = (
            days_passed,
            prices.loc[day],
            vert_barrier,
            top_barrier,
            bottom_barrier,
        )

    return barriers


def get_labels(barriers):
    """
    Assigns labels to each data point based on the triple-barrier method.
    Labels are: 0 (stop loss), 1 (no event), 2 (profit taking).
    """
    barriers["label_barrier"] = None
    for i in range(len(barriers.index)):
        start = barriers.index[i]
        end = barriers.vert_barrier[i]
        if pd.notna(end):

            # # assign the initial and final price
            price_initial = barriers.price[start]
            price_final = barriers.price[end]

            # assign the top and bottom barriers
            top_barrier = barriers.top_barrier[i]
            bottom_barrier = barriers.bottom_barrier[i]

            # print(top_barrier, bottom_barrier)

            # set the profit taking and stop loss conditions
            # condition_pt = (barriers.price[start: end] >= top_barrier).any()
            # condition_sl = (barriers.price[start: end] <= bottom_barrier).any()

            condition_pt = (barriers.price[start: end]).any() >= top_barrier
            condition_sl = (barriers.price[start: end]).any() <= bottom_barrier

            print(condition_pt, condition_sl)
            # assign the labels
            if condition_pt:
                barriers["label_barrier"][i] = 2
            else:
                if condition_sl:
                    barriers["label_barrier"][i] = 0
                else:
                    barriers["label_barrier"][i] = 1
                    
    return barriers


# Use function to produce barriers
barriers = get_barriers(prices, daily_volatility, t_final, upper_lower_multipliers)
# print(barriers.tail(5))
get_labels(barriers)

# Merge the barriers with the main dataset and drop the last t_final + 1 barriers (as they are too close to the end)
# print(barriers.tail(5))
# print(ohlcv.head(5))
# data_ohlcv = ohlcv.merge(barriers[['vert_barrier', 'top_barrier', 'bottom_barrier', 'label_barrier']], left_on='Date', right_on='Date')
# data_ohlcv.drop(data_ohlcv.tail(t_final + 1).index,inplace = True)
# data_ohlcv.tail(5)

print((barriers['label_barrier']).tolist())

[*********************100%***********************]  1 of 1 completed


<ipython-input-177-3325d414c4c2>:49: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for day, vol in daily_volatility.iteritems():


False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
Fals

In [146]:
bar = (data_ohlcv['label_barrier']).tolist()
print(bar)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

https://github.com/AI4Finance-Foundation/FinRL-Tutorials/blob/master/2-Advance/Crypto_Feature_Importance.ipynb

2.2 Adding Path Dependency: Triple-Barrier Method
To better incorporate the stop-loss and take-profit scenarios of a hypothetical trading strategy, we will modify the fixed-horizon labeling method so that it reflects which barrier has been touched first — upper, lower, or horizon. Hence the name: the triple-barrier method.

The labeling schema is defined as follows:

y = 2 : top barrier is hit first
y = 1 : right barrier is hit first
y = 0 : bottom barrier is hit first
What about the side of the bet?

The schema above works fine for long-only strategies, however things get more complicated when we allow for both long and short bets. If we are betting short, our profit/loss is inverted relative to the price action — we profit if the price goes down and we lose when the price goes up.

In order to account for this, we can simply represent side as 2 for long and 0 for short. Thus we can multiply our returns by the side, so whenever we’re betting short the negative returns become positive and vice-versa. Effectively, we flip the y = 2 and y = 0 labels if side = 0.

Let’s take a shot at the implementation (based on MLDP’s code). First, we define the procedure for getting the timestamps of the horizon barriers:

In [179]:

def get_vol(prices, span=100):
    # 1. compute returns of the form p[t]/p[t-1] - 1
    df0 = prices.pct_change()
    # 2. estimate rolling standard deviation
    df0 = df0.ewm(span=span).std()
    return df0

data_ohlcv = ohlcv.assign(volatility=get_vol(ohlcv.Close)).dropna()
data_ohlcv.head(10)  

,Open,High,Low,Close,Adj Close,Volume,volatility
Date,,,,,,,
2020-01-06 00:00:00-05:00,73.447502,74.989998,73.187500,74.949997,73.314880,118387200,0.012509
2020-01-07 00:00:00-05:00,74.959999,75.224998,74.370003,74.597504,72.970085,108872000,0.009089
2020-01-08 00:00:00-05:00,74.290001,76.110001,74.290001,75.797501,74.143913,132079200,0.011792
2020-01-09 00:00:00-05:00,76.809998,77.607498,76.550003,77.407501,75.718788,170108400,0.013231
2020-01-10 00:00:00-05:00,77.650002,78.167503,77.062500,77.582497,75.889954,140644800,0.011903
2020-01-13 00:00:00-05:00,77.910004,79.267502,77.787498,79.239998,77.511299,121532000,0.012393
2020-01-14 00:00:00-05:00,79.175003,79.392502,78.042503,78.169998,76.464630,161954400,0.013862
2020-01-15 00:00:00-05:00,77.962502,78.875000,77.387497,77.834999,76.136940,121923600,0.013309
2020-01-16 00:00:00-05:00,78.397499,78.925003,78.022499,78.809998,77.090668,108829200,0.012790


In [181]:
# Create function to obtain the barrier hits
def get_barriers():
  #create a container
  barriers = pd.DataFrame(columns=['days_passed', 
            'price', 'vert_barrier', \
            'top_barrier', 'bottom_barrier'], \
              index = daily_volatility.index)
  for day, vol in daily_volatility.iteritems():
    days_passed = len(daily_volatility.loc \
                  [daily_volatility.index[0] : day])
    #set the vertical barrier 
    if (days_passed + t_final < len(daily_volatility.index) \
        and t_final != 0):
        vert_barrier = daily_volatility.index[
                            days_passed + t_final]
    else:
        vert_barrier = np.nan
    #set the top barrier
    if upper_lower_multipliers[0] > 0:
        top_barrier = prices.loc[day] + prices.loc[day] * \
                      upper_lower_multipliers[0] * vol
    else:
        #set it to NaNs
        top_barrier = pd.Series(index=prices.index)
    #set the bottom barrier
    if upper_lower_multipliers[1] > 0:
        bottom_barrier = prices.loc[day] - prices.loc[day] * \
                      upper_lower_multipliers[1] * vol
    else: 
        #set it to NaNs
        bottom_barrier = pd.Series(index=prices.index)
        
    barriers.loc[day, ['days_passed', 'price', 'vert_barrier','top_barrier', 'bottom_barrier']] = \
    days_passed, prices.loc[day], vert_barrier, top_barrier, bottom_barrier

  return barriers

# Set barrier parameters

daily_volatility = data_ohlcv['volatility']
t_final = 25
upper_lower_multipliers = [2, 2]
price = data_ohlcv['Close']
prices = price[daily_volatility.index]

barriers = get_barriers()
barriers

<ipython-input-181-abc5e1b8946c>:8: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for day, vol in daily_volatility.iteritems():


,days_passed,price,vert_barrier,top_barrier,bottom_barrier
Date,,,,,
2020-01-06 00:00:00-05:00,1,74.949997,2020-02-12 00:00:00-05:00,76.825084,73.07491
2020-01-07 00:00:00-05:00,2,74.597504,2020-02-13 00:00:00-05:00,75.953563,73.241444
2020-01-08 00:00:00-05:00,3,75.797501,2020-02-14 00:00:00-05:00,77.585053,74.009948
2020-01-09 00:00:00-05:00,4,77.407501,2020-02-18 00:00:00-05:00,79.455833,75.35917
2020-01-10 00:00:00-05:00,5,77.582497,2020-02-19 00:00:00-05:00,79.42941,75.735583
...,...,...,...,...,...
2022-04-26 00:00:00-04:00,582,156.800003,NaN,162.541197,151.058809
2022-04-27 00:00:00-04:00,583,156.570007,NaN,162.245795,150.89422
2022-04-28 00:00:00-04:00,584,163.639999,NaN,169.880593,157.399406


2.3 Function to get label for the dataset (0, 1, 2)
<ul>
<li>0: hit the stoploss
<li>1: hit the time out
<li>2: hit the profit take
</ul>
The part in this function (commented out), allows for easy conversion to a regression analysis (currently it is classification). If one changes the labels to (-1, 0, 1), and change the hit on the vertical barrier to the function stated below.

That will make hitting the profit take barrier 1, the vertical barrier a range from (-1, 1), and the stoploss barrier -1. This is a continuos space then.

barriers['out'][i] = max(
          [(price_final - price_initial)/ 
            (top_barrier - price_initial), \
            (price_final - price_initial)/ \
            (price_initial - bottom_barrier)],\
            key=abs)

In [182]:
def get_labels():
  '''
  start: first day of the window
  end:last day of the window
  price_initial: first day stock price
  price_final:last day stock price
  top_barrier: profit taking limit
  bottom_barrier:stop loss limt
  condition_pt:top_barrier touching conditon
  condition_sl:bottom_barrier touching conditon
  '''

  barriers["label_barrier"] = None
  for i in range(len(barriers.index)):
    start = barriers.index[i]
    end = barriers.vert_barrier[i]
    if pd.notna(end):

        # assign the initial and final price
        price_initial = barriers.price[start]
        price_final = barriers.price[end]

        # assign the top and bottom barriers
        top_barrier = barriers.top_barrier[i]
        bottom_barrier = barriers.bottom_barrier[i]

        #set the profit taking and stop loss conditons
        condition_pt = (barriers.price[start: end] >= \
          top_barrier).any()
        condition_sl = (barriers.price[start: end] <= \
          bottom_barrier).any()

        #assign the labels
        if condition_pt: 
            barriers['label_barrier'][i] = 2
        elif condition_sl: 
            barriers['label_barrier'][i] = 0    
        else: 

          # Change to regression analysis by switching labels (-1, 0, 1)
          # and uncommenting the alternative function for vert barrier

          barriers['label_barrier'][i] = 1
            # barriers['label_barrier'][i] = max(
            #           [(price_final - price_initial)/ 
            #             (top_barrier - price_initial), \
            #             (price_final - price_initial)/ \
            #             (price_initial - bottom_barrier)],\
            #             key=abs)

  return


In [186]:

# Use function to produce barriers

get_labels()
barriers

# Merge the barriers with the main dataset and drop the last t_final + 1 barriers (as they are too close to the end)

data_ohlcv = data_ohlcv.merge(barriers[['vert_barrier', 'top_barrier', 'bottom_barrier', 'label_barrier']], left_on='Date', right_on='Date')
data_ohlcv.drop(data_ohlcv.tail(t_final + 1).index,inplace = True)
data_ohlcv.tail(25)

,Open,High,Low,Close,Adj Close,Volume,volatility,vert_barrier_x,top_barrier_x,bottom_barrier_x,label_barrier_x,vert_barrier_y,top_barrier_y,bottom_barrier_y,label_barrier_y,vert_barrier,top_barrier,bottom_barrier,label_barrier
Date,,,,,,,,,,,,,,,,,,,
2021-12-03 00:00:00-05:00,164.020004,164.960007,159.720001,161.839996,160.660812,118023100,0.013729,2022-01-11 00:00:00-05:00,166.283898,157.396095,2,2022-01-11 00:00:00-05:00,166.283898,157.396095,2,2022-01-11 00:00:00-05:00,166.283898,157.396095,2
2021-12-06 00:00:00-05:00,164.289993,167.880005,164.279999,165.320007,164.115479,107497000,0.013867,2022-01-12 00:00:00-05:00,169.904889,160.735126,2,2022-01-12 00:00:00-05:00,169.904889,160.735126,2,2022-01-12 00:00:00-05:00,169.904889,160.735126,2
2021-12-07 00:00:00-05:00,169.080002,171.580002,168.339996,171.179993,169.932755,120405400,0.014492,2022-01-13 00:00:00-05:00,176.141462,166.218523,2,2022-01-13 00:00:00-05:00,176.141462,166.218523,2,2022-01-13 00:00:00-05:00,176.141462,166.218523,2
2021-12-08 00:00:00-05:00,172.130005,175.960007,170.699997,175.080002,173.804382,116998900,0.014614,2022-01-14 00:00:00-05:00,180.197186,169.962818,2,2022-01-14 00:00:00-05:00,180.197186,169.962818,2,2022-01-14 00:00:00-05:00,180.197186,169.962818,2
2021-12-09 00:00:00-05:00,174.910004,176.750000,173.919998,174.559998,173.288162,108923700,0.014495,2022-01-18 00:00:00-05:00,179.620654,169.499341,2,2022-01-18 00:00:00-05:00,179.620654,169.499341,2,2022-01-18 00:00:00-05:00,179.620654,169.499341,2
2021-12-10 00:00:00-05:00,175.210007,179.630005,174.690002,179.449997,178.142532,115402700,0.014765,2022-01-19 00:00:00-05:00,184.749152,174.150842,0,2022-01-19 00:00:00-05:00,184.749152,174.150842,0,2022-01-19 00:00:00-05:00,184.749152,174.150842,0
2021-12-13 00:00:00-05:00,181.119995,182.130005,175.529999,175.740005,174.459564,153237000,0.015012,2022-01-20 00:00:00-05:00,181.016326,170.463685,2,2022-01-20 00:00:00-05:00,181.016326,170.463685,2,2022-01-20 00:00:00-05:00,181.016326,170.463685,2
2021-12-14 00:00:00-05:00,175.250000,177.740005,172.210007,174.330002,173.059845,139380400,0.014946,2022-01-21 00:00:00-05:00,179.540964,169.119039,2,2022-01-21 00:00:00-05:00,179.540964,169.119039,2,2022-01-21 00:00:00-05:00,179.540964,169.119039,2
2021-12-15 00:00:00-05:00,175.110001,179.500000,172.309998,179.300003,177.993607,131063300,0.015222,2022-01-24 00:00:00-05:00,184.758483,173.841523,0,2022-01-24 00:00:00-05:00,184.758483,173.841523,0,2022-01-24 00:00:00-05:00,184.758483,173.841523,0
